In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf

In [13]:
file_dir = 'Datos/RiesgoCABACABANA'
data = pd.read_csv(file_dir,sep=",",usecols=["incidenciaAcum14d"],quotechar='"',na_values=[''])
df = pd.DataFrame(data)
df = df.to_numpy()

In [15]:
model = keras.Sequential()
l1 = keras.layers.LSTM(100) 
l2 = keras.layers.LSTM(100)